### 處理word2vec文檔:
斷詞、編號、訓練

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
'''Initializaing data structure MSW'''
!gdown --id 1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a 

Downloading...
From: https://drive.google.com/uc?id=1_k1H_JyXwyRoRFcXa_YKGXgqxOg1Kx5a
To: /content/MSW.py
100% 642/642 [00:00<00:00, 893kB/s]


In [12]:
SEGDIR = "/content/drive/MyDrive/110-1ComSem_final/SenseWdictv2/SegTexts"
# SEGDIR裡的是沒有clean puncts和stopwords的版本
W2VDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_resources'
W2VSEGDIR = '/content/drive/MyDrive/110-1ComSem_final/w2v_SegTexts'
TFSEGDIR = '/content/drive/MyDrive/110-1ComSem_final/tf_SegTexts'

In [4]:
import json
import pickle
from MSW import MSW

In [7]:
# 幫wordㄉ詞加suffix 
# 定義sense的編號
with open (f'{W2VDIR}/MSW_leveled_list3.pkl', 'rb') as F:
    leveled_list = pickle.load(F)

In [8]:
leveled_list[0].senses

[{'category': 'main',
  'gloss': '八仙是道教中的八位神仙，分別代表男女老幼、貧賤富貴八種不同的人羣，由於八仙均為凡人得道，所以個性與百',
  'ismain': True,
  'link': 'https://zh.wikipedia.org/wiki/%E5%85%AB%E4%BB%99',
  'sense': '八仙',
  'senseidx': 1,
  'views': 491225},
 {'category': '淮南八仙',
  'gloss': '1 淮南八仙',
  'ismain': False,
  'link': '',
  'sense': '',
  'senseidx': 2,
  'views': 1},
 {'category': '地名',
  'gloss': '八仙裏，位於中華民國臺北市北投區',
  'ismain': False,
  'link': 'https://zh.wikipedia.org/wiki/%E5%85%AB%E4%BB%99%E9%87%8C',
  'sense': '八仙裏',
  'senseidx': 3,
  'views': -1},
 {'category': '地名',
  'gloss': '八仙鎮，位於中華人民共和國陝西省安康市平利縣',
  'ismain': False,
  'link': 'https://zh.wikipedia.org/wiki/%E5%85%AB%E4%BB%99%E9%95%87',
  'sense': '八仙鎮',
  'senseidx': 4,
  'views': 1},
 {'category': '地名',
  'gloss': '八仙嶺，位於香港新界大埔吐露港北岸',
  'ismain': False,
  'link': 'https://zh.wikipedia.org/wiki/%E5%85%AB%E4%BB%99%E5%B6%BA',
  'sense': '八仙嶺',
  'senseidx': 5,
  'views': 62988},
 {'category': '其他',
  'gloss': '八仙花，花名',
  'ismain': False,
  'link': '

In [ ]:
'''
for i in range(len(leveled_list)):
  msw = leveled_list[i]
  for j in range(len(msw.senses)):
    senseidx = (j+1)
    leveled_list[i].senses[j]['senseidx'] = senseidx
with open (f'{W2VDIR}/MSW_leveled_list3.pkl', 'wb') as F:
    pickle.dump(leveled_list, F)
'''

### 清sense文檔
存tf_SegTexts: clean stopwords和puncts

存w2v_SegTexts: clean puncts 保留前後dependent information 


In [ ]:
!wget https://github.com/Nana2929/stopwords/blob/master/cn_stopwords.txt?raw=true -O cn_stopwords.txt
!pip -q install opencc-python-reimplemented

--2021-12-20 11:48:18--  https://github.com/Nana2929/stopwords/blob/master/cn_stopwords.txt?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Nana2929/stopwords/raw/master/cn_stopwords.txt [following]
--2021-12-20 11:48:18--  https://github.com/Nana2929/stopwords/raw/master/cn_stopwords.txt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Nana2929/stopwords/master/cn_stopwords.txt [following]
--2021-12-20 11:48:18--  https://raw.githubusercontent.com/Nana2929/stopwords/master/cn_stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting

In [ ]:
# download stopwords
from opencc import OpenCC
cc = OpenCC('s2t')
stopwords = []
with open('./cn_stopwords.txt', 'r') as F:
  for line in F.readlines():
    t = cc.convert(line.strip('\n'))
    stopwords.append(t)
###超重要的stopwords!!!####
# 月份
stopwords = ' '.join(stopwords)+"這些 那些 的 更 有 這個 那個 他們 她們 它們 牠們 於 月 日 被"
stopwords = stopwords.split()

In [ ]:
import re 
punc = " ！？｡。＂＃＄％＆＇（()）＊＋，－／：；＜＝＞＠［＼］＾＿｀〈〉｛｜｝～«»《》｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."
punc+="\n"
punc+="\t"
def clean(token_list, rmstopwords):
  new_list = []
  for token in token_list:
    if rmstopwords:
      token = ''.join(x for x in list(token) if x not in punc and x not in stopwords)
    else:
      token = ''.join(x for x in list(token) if x not in punc)
    if len(token) != 0:
      new_list.append(token)
  return new_list

In [ ]:
def getsenselist(msw):
  return [f"{x['category']}_{x['sense']}" for x in msw.senses]

In [ ]:
'''
# word = '蘋果'
for msw in leveled_list:
    word = msw.word
    senseslist = getsenselist(msw)
    with open (f'{SEGDIR}/MSW_Segdict_{word}.json', 'r') as fp:
          word_dict = json.load(fp)
    print(word_dict)
    tf_worddict = {}
    w2v_worddict={}
    for sense in word_dict.keys():
        cleaned_sense  = [clean(line, rmstopwords = False) for line in word_dict[sense]]
        # for W2V segmented text
        w2v_worddict[sense] = cleaned_sense
        ccleaned_sense  = [clean(line, rmstopwords = True) for line in word_dict[sense]]
        tf_worddict[sense] = ccleaned_sense
    #  for TF segmented print(senseidx)     
    with open (f'{TFSEGDIR}/MSW_Segdict_{word}.json', 'w') as fp:
        json.dump(tf_worddict, fp, indent = 4, ensure_ascii = False)
    with open (f'{W2VSEGDIR}/MSW_Segdict_{word}.json', 'w') as fp:
        json.dump(w2v_worddict, fp, indent = 4, ensure_ascii = False)
'''

"\n# word = '蘋果'\nfor msw in leveled_list:\n    word = msw.word\n    senseslist = getsenselist(msw)\n    with open (f'{SEGDIR}/MSW_Segdict_{word}.json', 'r') as fp:\n          word_dict = json.load(fp)\n    print(word_dict)\n    tf_worddict = {}\n    w2v_worddict={}\n    for sense in word_dict.keys():\n        cleaned_sense  = [clean(line, rmstopwords = False) for line in word_dict[sense]]\n        # for W2V segmented text\n        w2v_worddict[sense] = cleaned_sense\n        ccleaned_sense  = [clean(line, rmstopwords = True) for line in word_dict[sense]]\n        tf_worddict[sense] = ccleaned_sense\n    #  for TF segmented print(senseidx)     \n    with open (f'{TFSEGDIR}/MSW_Segdict_{word}.json', 'w') as fp:\n        json.dump(tf_worddict, fp, indent = 4, ensure_ascii = False)\n    with open (f'{W2VSEGDIR}/MSW_Segdict_{word}.json', 'w') as fp:\n        json.dump(w2v_worddict, fp, indent = 4, ensure_ascii = False)\n"

### 幫w2v_SegTexts裡的所有該字都依照sense標上編號

In [ ]:
# 將leveled_list的資訊做成字典

In [ ]:
with open (f'{W2VDIR}/MSW_leveled_list3.pkl', 'rb') as F:
    leveled_list = pickle.load(F)

In [22]:
for msw in leveled_list:
    word = msw.word
    
    with open (f'{W2VSEGDIR}/MSW_Segdict_{word}.json', 'r') as fp:
        word_dict = json.load(fp)
    new_wd = {}
    # get senselist 
    for sense in msw.senses:
        senseid = sense['senseidx']
        reference = f"{sense['category']}_{sense['sense']}"
        # print(reference)
        # print(sense)
        
        # print(sensefile)
        try:
          sensefile = word_dict[reference]
        except: pass
        # plug in the suffix 
        n_reference = f"{sense['category']}_{senseid}"
        newsensefile = [[token+f'_{senseid}' if token == word else token for token in sent]
                        for sent in sensefile]
        new_wd[n_reference] = newsensefile
        # print('SEE:', newsensefile[0])
        # store the word_dict back 

    with open (f'{W2VSEGDIR}/MSW_Segdict_{word}_suff.json', 'w') as fp:
          # print('~~')
          json.dump(new_wd, fp, ensure_ascii = False, indent = 4)

### w2v訓練
全語料庫共90189句，
可以考慮gpu accel版本：https://github.com/SimonPavlik/word2vec-keras-in-gensim/tree/keras106
2. w2v:

  (1) model: CBOW > skip-gram

  (2) embedding-dim: 100 ~ 300為佳，越大越慢

  (3) iter: 訓練回數

  (4) min_count

  (5) worker: threading (parallel training)看要不要開大（越大越快）

  (6) context window (window_size): default = 5（左右各2），我覺好像也可以開大？


12/20 幹救命忘記切訓練和測試ㄌ

In [9]:
import random
ratio = 0.7 # Tfidf用0.5，因為越後面的文章越混亂，反而performance會下降
Train_Corpus = []
Test_Corpus = {} # Dict 

In [23]:
for msw in leveled_list:
    word = msw.word
    Test_Corpus[word] =[]
    with open (f'{W2VSEGDIR}/MSW_Segdict_{word}_suff.json', 'r') as fp:
        word_dict = json.load(fp)
        for sense in msw.senses:
            senseid = sense['senseidx']
            reference = f"{sense['category']}_{senseid}"
            try:
              sents = word_dict[reference]
              LEN = len(sents)
              Portion = round(ratio * LEN)
              print(f'{word}: ', LEN, Portion)
              # print(LEN, Portion)
              if Portion <=1: Portion = 1
              randomized = random.sample(range(0, LEN), Portion)
              for j in range(LEN):
                  if j in randomized:
                    Train_Corpus.append(sents[j]) 
                  else:
                    testpair = (sents[j], senseid)
                    if len(sents[j]) != 0:
                      if word == '荔枝': print(testpair)
                      Test_Corpus[word].append(testpair)
            except KeyError: 
              print(f'{reference} file does not exist.')
        del word_dict
# 90189句 in total 
print(len(Train_Corpus)) 
print(len(Test_Corpus)) 

八仙:  77 54
八仙:  77 54
八仙:  77 54
八仙:  77 54
八仙:  5 4
八仙:  14 10
八仙:  14 10
八仙:  77 54
巴比倫:  269 188
巴比倫:  269 188
巴比倫:  269 188
巴比倫:  269 188
巴比倫:  44 31
巴比倫:  5 4
巴比倫:  5 4
巴比倫:  6 4
巴比倫:  54 38
巴黎:  341 239
白宮:  52 36
白宮:  52 36
白宮:  52 36
白宮:  52 36
白宮:  52 36
白宮:  52 36
白宮:  52 36
白宮:  52 36
白宮:  5 4
白宮:  5 4
白宮:  21 15
半島:  3 2
半島:  47 33
半島:  13 9
半島:  13 9
半島:  13 9
半島:  13 9
半島:  56 39
半島:  56 39
半島:  56 39
半島:  56 39
半島:  4 3
別號:  13 9
別號:  8 6
彩虹:  138 97
彩虹:  27 19
彩虹:  99 69
彩虹:  6 4
彩虹:  101 71
彩虹:  101 71
彩虹:  10 7
彩虹:  4 3
彩虹:  6 4
彩虹:  8 6
彩虹:  1 1
彩虹:  39 27
彩虹:  3 2
彩虹:  13 9
赤道:  75 52
赤道:  5 4
赤道:  5 4
赤道:  5 4
赤道:  45 31
赤道:  4 3
赤道:  31 22
刺客:  10 7
刺客:  40 28
刺客:  4 3
刺客:  36 25
刺客:  20 14
刺客:  33 23
刺客:  33 23
刺客:  29 20
刺客:  28 20
刺客:  28 20
刺客:  5 4
大氣層:  51 36
大氣層:  51 36
大氣層:  47 33
大氣層:  47 33
大氣層:  14 10
淡水:  42 29
淡水:  9 6
淡水:  71 50
淡水:  13 9
淡水:  10 7
淡水:  18 13
淡水:  34 24
淡水:  12 8
淡水:  111 78
淡水:  85 59
淡水:  109 76
淡水:  22 15
淡水:  4 3
刀:  7 5
刀:  7 5


In [ ]:
# 看來只能手動改荔枝這邊ㄌ


In [ ]:
for msw in leveled_list:
    print(msw)

* word: 八仙,* #senses:11 *hasLevel: True
* word: 巴比倫,* #senses:11 *hasLevel: True
* word: 巴黎,* #senses:4 *hasLevel: True
* word: 白宮,* #senses:11 *hasLevel: True
* word: 半島,* #senses:11 *hasLevel: True
* word: 別號,* #senses:2 *hasLevel: True
* word: 彩虹,* #senses:14 *hasLevel: True
* word: 赤道,* #senses:8 *hasLevel: True
* word: 刺客,* #senses:12 *hasLevel: True
* word: 大氣層,* #senses:9 *hasLevel: True
* word: 淡水,* #senses:13 *hasLevel: True
* word: 刀,* #senses:9 *hasLevel: True
* word: 道,* #senses:11 *hasLevel: True
* word: 地獄,* #senses:6 *hasLevel: True
* word: 電信,* #senses:5 *hasLevel: True
* word: 鼎,* #senses:4 *hasLevel: True
* word: 東海,* #senses:20 *hasLevel: True
* word: 東京,* #senses:32 *hasLevel: True
* word: 豆蔻,* #senses:8 *hasLevel: True
* word: 毒液,* #senses:3 *hasLevel: True
* word: 獨立,* #senses:22 *hasLevel: True
* word: 段,* #senses:5 *hasLevel: True
* word: 盾,* #senses:29 *hasLevel: True
* word: 番茄,* #senses:15 *hasLevel: True
* word: 防火牆,* #senses:5 *hasLevel: True
* word: 粉末,* #

In [ ]:
rand = random.randint(0, min(len(Train_Corpus), len(Test_Corpus)))
print(rand)
print(Train_Corpus[rand])
for sent in Test_Corpus['蘋果']:
  print(sent)

In [ ]:
import pickle
with open(f'{W2VDIR}/Train_Corpus.pkl', 'wb') as f:
  pickle.dump(Train_Corpus, f)
with open(f'{W2VDIR}/Test_Corpus.pkl', 'wb') as f:
  pickle.dump(Test_Corpus, f)

In [ ]:
from gensim.models import Word2Vec
# embedding_dim = 200
EMBDIM= 200
model = Word2Vec(size = EMBDIM,
                window = 7,
                min_count = 3,
                workers = 8,
                batch_words = 10000)
                #sg = 1 : use skip-gram model

model.build_vocab(Train_Corpus)
model.train(Train_Corpus, total_examples=len(Train_Corpus), epochs = 30)

(96776652, 109024290)

In [ ]:
SAVEpath= f'{W2VDIR}/TrainedC_embeddings.wv'
model.save(SAVEpath)
# model.wv['八仙_1']

In [ ]:
similar_words = {
    search_term:
    [item[0] for item in model.wv.most_similar([search_term], topn=8)]
    for search_term in
    ['臺灣_1', '臺灣_2', '臺灣_3', '臺灣_4', '臺灣_5',
     '臺灣_6', '臺灣_7', '臺灣_8', '臺灣_9', '臺灣_10']
}
similar_words

{'臺灣_1': ['臺灣_2', '臺灣_16', '臺灣_6', '臺灣', '臺灣_3', '基隆', '臺北', '臺灣_4'],
 '臺灣_10': ['臺灣_9', '判斷', '臺獨', '案', '蔣家', '特朗普', '江南_17', '單方面'],
 '臺灣_2': ['臺灣_1', '臺灣_16', '臺灣_6', '臺灣', '臺灣_3', '基隆', '臺北', '臺灣_4'],
 '臺灣_3': ['臺灣_4', '中央山脈', '大漢溪', '木曾山脈', '地形', '山地', '地勢', '下莊'],
 '臺灣_4': ['臺灣_3', '地勢', '山地', '地形', '大漢溪', '亞熱帶', '木曾山脈', '中央山脈'],
 '臺灣_5': ['浮出', '寶雲山', '全副武裝', '志摩半島', '初島', '豢', '試航', '從頭'],
 '臺灣_6': ['臺灣_2', '臺灣_13', '臺灣_1', '嘉義縣', '彰化縣', '臺灣_16', '臺南', '板南線'],
 '臺灣_7': ['臺灣_8',
  '臺灣_12',
  '省委',
  '劉格平爲',
  '同胞書',
  '張聞天',
  '分裂出去',
  'ProvinceofChina'],
 '臺灣_8': ['臺灣_7', '臺灣_12', '省委', '國務院', '同胞書', '下屬', '部級', '劉格平爲'],
 '臺灣_9': ['臺灣_10', '民進黨', '蔡英文', '法理', '獨立_11', '踏實', '意旨', '蔣家']}

In [ ]:
similar_words = {
    search_term:
    [item[0] for item in model.wv.most_similar([search_term], topn=5)]
    for search_term in
    ['怪物_1', '怪物_2', '怪物_3', '怪物_4', '怪物_5',
     '怪物_6', '怪物_7', '怪物_8','怪物_9', '怪物_10', '怪物_11']
}
similar_words

{'怪物_1': ['描寫成', '死神_9', '死神_1', '大戟', '死神_7'],
 '怪物_10': ['怪物_8', '怪物_9', '醫學系', '智勝', '視死如歸'],
 '怪物_11': ['Part2', '典藏版', '狂飆', '喬星', '街機版'],
 '怪物_2': ['怪物_3', 'TCM', '鐵東區', '新址', '鞍山市'],
 '怪物_3': ['怪物_2', '淩霄花', 'Robotmasters', '交誼典·', '賽菈傑利亞'],
 '怪物_4': ['普羅米修斯', '屍變成', '活死', '認識', '深淵'],
 '怪物_5': ['駭人', '邪神', '賢書', '火雲', '金剛_17'],
 '怪物_6': ['玫瑰果', '省華縣', '820萬', 'EAFUS', '增長到'],
 '怪物_7': ['第2000', '主序帶', '學校區', 'NAVER', 'OfficeMojo'],
 '怪物_8': ['怪物_10', '怪物_9', '彌爾頓', '詹姆士·麥迪遜', '出人頭地'],
 '怪物_9': ['怪物_8', '怪物_10', '亞歷西斯·托克維爾', '出人頭地', '智勝']}